# Mi proyecto en Python

Es un diccionario personalizado. Quiero subirlo a [mi portafolio en github](https://github.com/losataschu).

Primero defino la función que me ayuadará a determinar si acaso una entrada tiene una, dos o más palabras.
Testeé esa función con la lista `mis palabras`:

In [5]:
mis_palabras = ['abarbeiten', 'die Wertschöpfung', 'etwas in Worte fassen']

def contarPalabras(texto):
  if len(texto) == 0:
    return 0
  palabras = 1
  for caracter in texto:
    if caracter == ' ' and texto[0] != caracter and texto[-1] != caracter:
      palabras += 1
  return palabras


Luego voy a adjuntar los archivos que tengo el drive:

In [5]:
# solo para drive
# from google.colab import drive
# drive.mount('/content/drive')

Las librerías que uso para crear mi diccionario: `json` y `pandas`.

In [7]:
import json
import pandas as pd

## Resumen de mis clases

1. Entrada
    1. Texto
    5. Fecha
    4. Categoría
3. Sustantivo
    * Artículo
50. Diccionario
    + Se crea a partir de las entradas
    + Puede agregar entradas
    + Debe permitir mostrar sublistas filtrando por diferentes criterios

Tengo una clase global para una entrada en mi diccionario, `Entrada`, que consiste de un texto, una fecha y una categoría.
La subclase `Sustantivo` además tiene el artículo que lo diferencia del resto de las entradas, que van a ser palabras solas o frases compuestas:

In [9]:
#Codigo Entrada
class Entrada:
  def __init__(self, texto, fecha, categoria):
    self.texto = texto
    self.fecha = fecha
    self.categoria = categoria
  def __str__(self):
    return self.texto
  def __repr__(self):
    return f"{self.texto}"

#Codigo Sustantivo
class Sustantivo(Entrada):
  def __init__(self, articulo, texto, fecha, categoria):
    super().__init__(texto, fecha, categoria)
    self.articulo = articulo
  def __repr__(self):
    return f"{self.articulo} {self.texto}"



In [11]:
# Codigo Clase Diccionario
class Diccionario:
  def __init__(self, direccion):
    self.direccion = direccion
    self.listado = self.crear_listado()

  def crear_listado(self):
    with open(self.direccion, 'r') as file:
      mis_entradas = json.loads(file.read())
      #print(mis_entradas)
      entradas_con_categoria = []
      for entrada in mis_entradas["Entradas"]:
        #print(entrada["palabra"])
        if contarPalabras(entrada["palabra"]) == 2:
          entradas_con_categoria.append(Sustantivo(articulo=entrada["palabra"][0:3], texto=entrada["palabra"][4:], fecha=entrada["fecha"], categoria="Sustantivo"))
        else:
          entradas_con_categoria.append(Entrada(texto=entrada["palabra"], fecha=entrada["fecha"], categoria="Otra"))
      return entradas_con_categoria


In [13]:
class Tabla:
  def __init__(self, listado):
    self.listado = listado
    self.tabla = self.crear_tabla()
    
# Crear una tabla ocupando las entradas del diccionario
  def crear_tabla(self):
    enumeracion = []
    coleccion = []
    i = 1
    entrada_i = "Entrada "
    for entrada in self.listado:
      enumeracion.append(entrada_i + str(i))
      # TODO: agregar una excepcion para regular errores
      fecha_legible = f'{entrada.fecha[0]}-{entrada.fecha[1]}-{entrada.fecha[2]}'
      linea = [entrada.__repr__(), fecha_legible, entrada.categoria]
      coleccion.append(linea)
      i += 1
    return pd.DataFrame(coleccion, index=enumeracion, columns=["Palabra", "Fecha", "Categoria"])


In [45]:
def isSustantivo(entrada):
    if contarPalabras(entrada.texto) == 2:
      return true

# esta es la función que filtrará las tablas y entrega los resultados
def filtrarTabla(tabla, criterio, valor):
    match criterio:
      case "categoria": return filtrarCategoria(tabla, valor)
      case "letra_inicial": return filtrarLetraInicial(tabla, valor)
      case _: return "Para tal criterio no hay nada aun"

def filtrarCategoria(tabla, valor):
    tabla_filtrada = tabla.copy()
    match valor:
      case "Sustantivo" | "Otra": return tabla_filtrada.loc[lambda df: df["Categoria"] == valor, :]
      case _: return "Categoría inválida."

def filtrarLetraInicial(tabla, valor):
    tabla_filtrada = tabla.copy()
    if len(valor) != 1:
      return "Ingresar solamente una letra."
    valor = valor.lower()
    for i in tabla.loc[:,"Palabra"]:
      if contarPalabras(i) == 2 and i[4] != valor.capitalize():
        idx = tabla.index[tabla["Palabra"]==i].tolist()
        tabla_filtrada = tabla_filtrada.drop(idx)
      elif contarPalabras(i) == 1 and i[0] != valor:
        idx = tabla.index[tabla["Palabra"]==i].tolist()
        tabla_filtrada = tabla_filtrada.drop(idx)
    return tabla_filtrada


In [47]:
nuevo_diccionario = Diccionario("entradas_reducidas.json")
mis_entradas = nuevo_diccionario.listado
print(mis_entradas)
#print(nuevo_diccionario.listado[0].fecha)
#print(nuevo_diccionario.listado[0].__str__())
#print(nuevo_diccionario.listado[1].__repr__())
#print(nuevo_diccionario.listado[1].articulo)

[abarbeiten, abfangen, die Abfolge, abgelten, abgrenzen, die Besteuerung, bestmoeglich, bewirten, durchreichen, einheitlich, einordnen, der Einsatz, einsortieren, entbehrlich, das Entgelt, entkoppeln, das Fachkonzept, fachlich, die Fachlichkeit, die Festlegung, das Formelwerk, foermlich, gegenueberstellen, misslingen, nachgelagert, nahelegen, niederdruecken, notduerftig, die Plausibilisierung, raushauen, die Rechnungslegung, die Rentenperiode, das Tafelwerk, ungemein, unterlassen, unuebersichtlich, unzulaenglich, verbindlich, verkomplizieren, verschaffen, vervollstaendigen, die Verweildauer, der Vorgang, das Vorhaben, vorlaeufig, der Zeitwert, zusammensetzen, der Zusatz, die Zustaendigkeit, die Zusicherung, zuweisen, zwangslaeufig]


In [49]:
nueva_tabla = Tabla(mis_entradas)
#print(nueva_tabla.tabla)
tabla_entradas = nueva_tabla.tabla
encabezado = tabla_entradas.head(10)
print(encabezado)

                    Palabra       Fecha   Categoria
Entrada 1        abarbeiten  29-11-2022        Otra
Entrada 2          abfangen   24-5-2023        Otra
Entrada 3       die Abfolge  12-12-2022  Sustantivo
Entrada 4          abgelten  28-12-2022        Otra
Entrada 5         abgrenzen   24-5-2023        Otra
Entrada 6   die Besteuerung   7-12-2022  Sustantivo
Entrada 7      bestmoeglich  13-12-2022        Otra
Entrada 8          bewirten  29-12-2022        Otra
Entrada 9      durchreichen  13-12-2022        Otra
Entrada 10      einheitlich  21-12-2022        Otra


In [51]:
filtrarTabla(tabla_entradas, "letra_inicial", "F")

,Palabra,Fecha,Categoria
Entrada 17,das Fachkonzept,12-12-2022,Sustantivo
Entrada 18,fachlich,12-12-2022,Otra
Entrada 19,die Fachlichkeit,21-12-2022,Sustantivo
Entrada 20,die Festlegung,12-12-2022,Sustantivo
Entrada 21,das Formelwerk,13-12-2022,Sustantivo
Entrada 22,foermlich,29-11-2022,Otra


## Próximos pasos:

- Filtrar según criterios:
  1. Alfabético
  2. Por fecha
  3. ~~Por categoría~~
- A~nadir palabras al archivo JSON.
- ~~¿Crear una nueva clase *Tabla*?~~

## Ideas adicionales:
+ Crear el archivo JSON con pandas en mi clase `Diccionario` y crear una tabla que se pueda visualizar mejor.

In [157]:
pd.read_json('entrada_simple.json', orient="records")

,entrada,fecha
0,abarbeiten,"[29, 11, 2022]"
1,die Abfolge,"[12, 12, 2022]"


In [22]:
import numpy as np

index = pd.date_range("1/1/2000", periods=8)

df = pd.DataFrame(np.random.randn(8, 3), index=index, columns=["A", "B", "C"])
print(np.random.randn(8, 3) )
print(df)

[[ 9.41910128e-01  7.90234512e-01 -2.18467175e-01]
 [-7.96855617e-01 -3.84789061e-01 -4.97087591e-01]
 [ 1.49431316e+00  7.57519435e-01 -5.85202887e-01]
 [ 3.05734161e-01 -4.19143336e-02  1.74862926e+00]
 [ 1.00568524e+00 -6.81190402e-04 -2.55177632e-01]
 [ 4.69615209e-01 -9.56371756e-01  2.06806276e+00]
 [-9.71785672e-01 -1.55651529e-01 -3.01620397e-01]
 [ 1.36386970e-01 -2.81418896e-01 -3.89706242e-01]]
                   A         B         C
2000-01-01 -2.759640 -0.312461 -0.548958
2000-01-02  1.206670  1.501298  2.315188
2000-01-03  0.849976  0.897112  0.074638
2000-01-04 -0.547975  2.160294 -0.046115
2000-01-05  1.968564 -0.067033 -0.363519
2000-01-06  0.062446  0.282341  0.233524
2000-01-07 -0.612508  0.259587  0.378447
2000-01-08  0.398564  0.736146 -1.216213


In [66]:
def reordenar(lista):
  lista[-1] = -5
  return lista

In [68]:
lista = [1,2,3,4]
reordenar(lista)
print(lista)

[1, 2, 3, -5]


In [181]:
p = "ab"
len(p)

2